In [8]:
import json
import pandas as pd
import numpy as np
import wget
import os
import zipfile
import json
from collections import ChainMap
import tqdm
from config import required_col,rename_col

In [4]:
def check_download_format(n:int)->pd.DataFrame:
    """Function to process and download clinicaltrial Gov Files,Currenty it process n requests

    Returns:
        pd.DataFrame: Selected columns from clincalTrial Gov
    """
    if 'AllAPIJSON.zip' not in os.listdir():    
        url = 'https://ClinicalTrials.gov/AllAPIJSON.zip'
        filename = wget.download(url)
    else:
        print('file already present')
    zip = zipfile.ZipFile('AllAPIJSON.zip', "r")
    all_trials = [f for f in zip.namelist() if f.endswith('.json')]
    print('Total Number of Trials is '+str(len(all_trials)))
    ## Reading
    all_files = pd.DataFrame()
    for f in tqdm.tqdm(all_trials[:n]):
        with zip.open(f) as file:
            data_file = file.read()
            d = json.loads(data_file.decode("utf-8"))
            data_file_pd = pd.json_normalize(d)
            all_files = pd.concat([all_files,data_file_pd],ignore_index=True)
    list_col = []
    for col in all_files.columns:
        if any(isinstance(f,list) for f in all_files[col].tolist()):
            list_col.append(col)
    return all_files[[col for col in all_files.columns if col not in list_col]]

In [5]:
selected_trials = check_download_format(n=1000)

file already present
Total Number of Trials is 430807


100%|██████████| 1001/1001 [00:03<00:00, 268.39it/s]


In [9]:
selected_trials = selected_trials[required_col]

In [10]:
selected_trials = selected_trials.rename(rename_col,axis = 1)

In [11]:
selected_trials.to_csv('Select_trials.csv',index=False)